### Secuencias genéticas de los diferentes nimales

In [1]:
CABALLO_A_2  = "ATGGTGCTGTCTGCCGCCGACAAGACCAACGTCAAGGCCGCCTGGAGTAAGGTTGGCGGCCACGCTGGCGAGTATGGCGCAGAGGCCCTAGAGAGGATGTTCCTGGGCTTCCCCACCACCAAGACCTACTTCCCCCACTTCGATCTGAGCCACGGCTCCGCCCAGGTCAAGGCCCACGGCCAGAAGGTGGGCGACGCGCTGACTCTCGCCGTGGGCCACCTGGACGACCTGCCTGGCGCCCTGTCGAATCTGAGCGACCTGCACGCACACAAGCTGCGCGTGGACCCCGTCAACTTCAAGCTCCTGAGTCATTGCCTGCTGTCCACCTTGGCCGTCCACCTCCCCAACGATTTCACCCCTGCCGTCCACGCCTCCCTGGACAAGTTCTTGAGCAGTGTGAGCACCGTGCTGACCTCCAAATACCGTTAA"
CABRA_A_I    = "ATGGTGCTGTCTGCCGCCGACAAGTCCAATGTCAAGGCCGCCTGGGGCAAGGTTGGCGGCAACGCTGGAGCTTATGGCGCAGAGGCTCTGGAGAGGATGTTCCTGAGCTTCCCCACCACCAAGACCTACTTCCCCCACTTCGACCTGAGCCACGGCTCGGCCCAGGTCAAGGGCCACGGCGAGAAGGTGGCCGCCGCGCTGACCAAAGCGGTGGGCCACCTGGACGACCTGCCCGGTACTCTGTCTGATCTGAGTGACCTGCACGCCCACAAGCTGCGTGTGGACCCGGTCAACTTTAAGCTTCTGAGCCACTCCCTGCTGGTGACCCTGGCCTGCCACCTCCCCAATGATTTCACCCCCGCGGTCCACGCCTCCCTGGACAAGTTCTTGGCCAACGTGAGCACCGTGCTGACCTCCAAATACCGTTAA"
GALLINA_A_D  = "ATGCTGACTGCCGAGGACAAGAAGCTCATCCAGCAGGCCTGGGAGAGGGCCGCTTCCCACCAGGAGGAGTTTGGAGCTGAGGCTCTGACTAGGATGTTCACCACCTATCCCCAGACCAAGACCTACTTCCCCCACTTCGACCTTTCGCCTGGCTCTGACCAGGTCCGTGGCCATGGCAAGAAGGTGTTGGGTGCCCTGGGCAACGCCGTGAAGAACGTGGACAACCTCAGCCAGGCCATGGCTGAGCTGAGCAACCTGCATGCCTACAACCTGCGTGTTGACCCCGTCAATTTCAAGCTGTTGTCGCAGTGCATCCAGGTGGTGCTGGCTGTACACATGGGCAAAGACTACACCCCTGAAGTGCATGCTGCCTTCGACAAGTTCCTGTCTGCCGTGTCTGCTGTGCTGGCTGAGAAGTACAGATAA"

CABALLO_MOD  = "ATGGTGCTGTCTGCCGCCGACAAGACCAACGTCAAGGCCGCCTGGAGTAAGGTTGGCGGCCACGCTGGCGAGTATGGCGCAGAGGCCCTAGAGAGGATGTTCCTGGGCTTCCCCACCACCAAGACCTACTTCCCCCACTTCGATCTGAGCCACGGCTCCGCCCAGGTCAAGGCCCACGGCCAGAAGGTGGGCGACGCGCTGACTCTCGCCGTGGGCCACCTGGACGACCTGCCTGGCGCCCTGTCGAATCTGAGCGACCTGCACGCACACAAGCTGCGCGTGGACCCCGTCAACTTCAAGCTCCTGAGTCATTGCCTGCTGTCCACCTTGGCCGTCCACCTCCCCAACGATTTCACCCCTGCCGTCCTCGCCTCCCTGGACAAGTTCTTGAGCAGTGTGAGCACCGTGCTGACCTCCAAATACCGTTAA"

### Longitudes de las secuencias genéticas

In [2]:
print(len(CABALLO_A_2))
print(len(CABRA_A_I))
print(len(GALLINA_A_D))
print(len(CABALLO_MOD))

429
429
426
429


### Librerias necesarias para la simulación

In [3]:
import sys
import numpy as np
import math

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer, IBMQ
from qiskit.visualization import plot_histogram, plot_bloch_multivector

from qiskit.extensions import Initialize

### Circuito para construir la superposición cuántica del bitstring

In [4]:
def encode_bitstring(bitstring, qr, cr, inverse=False):
    
    n = math.ceil(math.log2(len(bitstring))) +  2                #number of qubits
    assert n > 4, "the length of bitstring must be at least 2"
    
    qc = QuantumCircuit(qr, cr)
    
    #the probability amplitude of the desired state
    desired_vector = np.array([ 0.0 for i in range(2**n) ])     #initialize to zero
    
    qc_init = QuantumCircuit(n) # Creación de compuertas circuitales para inicialización
    inverse_qc_init = QuantumCircuit(n) # Creación de compuertas circuitales para inversión de la inicialización

    amplitude = np.sqrt(1.0/len(bitstring))
    
    for i, b in enumerate(bitstring):
        pos = i * 4
        # print(i)
        # print(b)
        # print(pos)
        if b == "A":
            desired_vector[pos] = amplitude
        elif b == "C":
            desired_vector[pos+1] = amplitude
        elif b == "G":
            desired_vector[pos+2] = amplitude
        elif b == "T":
            desired_vector[pos+3] = amplitude
        # print(pos)
        
    # print(desired_vector[:50])
    # print(len(desired_vector))
    # print(sum((desired_vector)**2))

    qc_init = Initialize(desired_vector)
   
    if not inverse:

        qc.append(qc_init, qr)  #  
    
        qc.barrier(qr)
        
    else:

        inverse_qc_init = qc_init.gates_to_uncompute()
        qc.append(inverse_qc_init, qr)
        
        qc.barrier(qr)
        for i in range(n):
            qc.measure(qr[i], cr[i])
    print()
    return qc

### Se crean los circuitos cuánticos para los estados de los códigos genéticos

In [5]:
n = math.ceil(math.log2(len(CABALLO_A_2))) + 2                 #number of qubits
# print(n)
qr = QuantumRegister(n)
cr = ClassicalRegister(n)

qc_caballo  = encode_bitstring(CABALLO_A_2, qr, cr)
qc_cabra    = encode_bitstring(CABRA_A_I, qr, cr)
qc_gallina  = encode_bitstring(GALLINA_A_D, qr, cr)

qc_caballo_mod = encode_bitstring(CABALLO_MOD, qr, cr)

circs = {"CABALLO_A_2": qc_caballo, "CABRA_A_I": qc_cabra, "GALLINA_A_D": qc_gallina, "CABALLO_MOD": qc_caballo_mod}

### Se invierten los circuitos cuánticos

In [6]:
inverse_qc_caballo     = encode_bitstring(CABALLO_A_2, qr, cr, inverse=True)
inverse_qc_cabra = encode_bitstring(CABRA_A_I,   qr, cr, inverse=True)
inverse_qc_gallina = encode_bitstring(GALLINA_A_D, qr, cr, inverse=True)

inverse_qc_caballo_mod = encode_bitstring(CABALLO_MOD, qr, cr, inverse=True)

inverse_circs = {"CABALLO_A_2": inverse_qc_caballo, "CABRA_A_I": inverse_qc_cabra, "GALLINA_A_D": inverse_qc_gallina, "CABALLO_MOD": inverse_qc_caballo_mod}

### Representación gráfica de la inversión de los circuitos cuánticos

In [7]:
inverse_qc_caballo.draw()

┌────────────────┐ ░ ┌─┐                              
 q0_0: ┤0               ├─░─┤M├──────────────────────────────
       │                │ ░ └╥┘┌─┐                           
 q0_1: ┤1               ├─░──╫─┤M├───────────────────────────
       │                │ ░  ║ └╥┘┌─┐                        
 q0_2: ┤2               ├─░──╫──╫─┤M├────────────────────────
       │                │ ░  ║  ║ └╥┘┌─┐                     
 q0_3: ┤3               ├─░──╫──╫──╫─┤M├─────────────────────
       │                │ ░  ║  ║  ║ └╥┘┌─┐                  
 q0_4: ┤4               ├─░──╫──╫──╫──╫─┤M├──────────────────
       │                │ ░  ║  ║  ║  ║ └╥┘┌─┐               
 q0_5: ┤5  disentangler ├─░──╫──╫──╫──╫──╫─┤M├───────────────
       │                │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐            
 q0_6: ┤6               ├─░──╫──╫──╫──╫──╫──╫─┤M├────────────
       │                │ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐         
 q0_7: ┤7               ├─░──╫──╫──╫──╫──╫──╫──╫─┤M├─────────
       │                │ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      
 q0_8: ┤8               ├─░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────
       │                │ ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
 q0_9: ┤9               ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───
       │                │ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
q0_10: ┤10              ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
       └────────────────┘ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
c0: 11/══════════════════════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
                             0  1  2  3  4  5  6  7  8  9  10

### Comparar Caballo

In [8]:
from qiskit import IBMQ, BasicAer

key1 = "CABALLO_A_2"       #the name of the code used as key to find similar ones

key2 = "CABALLO_MOD"

# use local simulator
backend = BasicAer.get_backend("qasm_simulator")
shots = 8192

combined_circs = {}
count = {}

combined_circs[key2] = circs[key1].compose(inverse_circs[key2])   #combined circuits to look for similar codes
job = execute(combined_circs[key2], backend=backend,shots=shots)
st = job.result().get_counts(combined_circs[key2])
if "0"*n in st:
    sim_score = st["0"*n]/shots
else:
    sim_score = 0.0

print("Similaridad de Caballo-(Alpha-2) consigo misma modificando un nucleótido: ", sim_score)

Similaridad de Caballo-(Alpha-2) consigo misma modificando un nucleótido:  0.994384765625


### Comparar Caballo con Cabra

In [14]:
key1 = "CABALLO_A_2"
key2 = "CABRA_A_I"
key3 = "GALLINA_A_D"

# use local simulator
backend = BasicAer.get_backend("qasm_simulator")
shots = 8192

combined_circs = {}
count = {}

combined_circs[key2] = circs[key1].compose(inverse_circs[key2])   #combined circuits to look for similar codes
job = execute(combined_circs[key2], backend=backend,shots=shots)
st = job.result().get_counts(combined_circs[key2])
if "0"*n in st:
    sim_score = st["0"*n]/shots
else:
    sim_score = 0.0
print("Similaridad entre Caballo-(Alpha-2) y Cabra-(Alpha-i): ", sim_score)


Similaridad entre Caballo-(Alpha-2) y Cabra-(Alpha-i):  0.779541015625


### Comparar Caballo con Gallina

In [15]:
combined_circs[key3] = circs[key1].compose(inverse_circs[key3])   #combined circuits to look for similar codes
job = execute(combined_circs[key3], backend=backend,shots=shots)
st = job.result().get_counts(combined_circs[key3])
if "0"*n in st:
    sim_score = st["0"*n]/shots
else:
    sim_score = 0.0
print("Similaridad entre Caballo-(Alpha-2) y Gallina-(Alpha-D): ", sim_score)

Similaridad entre Caballo-(Alpha-2) y Gallina-(Alpha-D):  0.08447265625


### Comparar Cabra con Gallina

In [16]:
combined_circs[key3] = circs[key2].compose(inverse_circs[key3])   #combined circuits to look for similar codes
job = execute(combined_circs[key3], backend=backend,shots=shots)
st = job.result().get_counts(combined_circs[key3])
if "0"*n in st:
    sim_score = st["0"*n]/shots
else:
    sim_score = 0.0
print("Similaridad entre Cabra-(Alpha-i) y Gallina-(Alpha-D): ", sim_score)

Similaridad entre Cabra-(Alpha-i) y Gallina-(Alpha-D):  0.095458984375
